# Feature Engineering
- Create Aggregate Features
- Extract Features
- Encode Categorical Variables
- Handle Missing Values
- Normalize/Standardize Numerical Features

In [95]:
import pandas as pd
import numpy as np
import importlib

import os
import sys

sys.path.append(os.path.abspath(os.path.join('..','src')))

import eda
importlib.reload(eda)

import feature_eng
importlib.reload(feature_eng)

import warnings
warnings.filterwarnings('ignore')

In [96]:
# intialize the feature engineering clas
fre_eng = feature_eng.FeatEng('data.csv')
fre_eng.change_datatype()

df = fre_eng.get_dataframe()

New shape of the dataframe: (95662, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   TransactionId         95662 non-null  category           
 1   BatchId               95662 non-null  category           
 2   AccountId             95662 non-null  category           
 3   SubscriptionId        95662 non-null  category           
 4   CustomerId            95662 non-null  category           
 5   ProviderId            95662 non-null  category           
 6   ProductId             95662 non-null  category           
 7   ProductCategory       95662 non-null  category           
 8   ChannelId             95662 non-null  category           
 9   Amount                95662 non-null  float64            
 10  Value                 95662 non-null  int64              
 11  TransactionStartTime  95662

In [97]:
# add total transaction feature
fre_eng.aggregate_transaction()

# extract each parts of datetime
fre_eng.extract_datetime()

(95662, 18)
Shape of the dataframe (95662, 22)


In [98]:
# change categorical features to numerical
fre_eng.label_encoder()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype              
---  ------                               --------------  -----              
 0   TransactionId                        95662 non-null  int64              
 1   BatchId                              95662 non-null  int64              
 2   AccountId                            95662 non-null  int64              
 3   SubscriptionId                       95662 non-null  int64              
 4   CustomerId                           95662 non-null  int64              
 5   ProviderId                           95662 non-null  int64              
 6   ProductId                            95662 non-null  int64              
 7   ProductCategory                      95662 non-null  int64              
 8   ChannelId                            95662 non-null  int64              
 9   Amount                      

In [99]:
# null value columns
fre_eng.null_col()

Column containing null value: Amount_std


In [100]:
# check
df = fre_eng.get_dataframe()
df[df['Amount_std'].isnull()]

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,...,PricingStrategy,FraudResult,Amount_Total,Amount_Average,TransactionId_number_of_Transaction,Amount_std,Month,Year,Day,Hour


- Amount_std is null for customers with only one transaction.